In [1]:
import numpy as np
from numba import jit

In [179]:
class HMM:
    def forward(A,B,pi,sequence):
        '''
        Perform the forward step in Baum-Welch Algorithm.
        
        Parameters
        ----------
        A: np.ndarray
            Stochastic transition matrix.
        B: np.ndarray
            Emission matrix.
        pi: np.1darray
            Initial state distribution.
        sequence: array-like
            The observed sequence.
            Need to be converted to integer coded.    
        '''
        N=A.shape[0]
        M=B.shape[1]
        T=len(sequence)
        alpha=np.zeros([T,N])
        alpha[0]=pi*B[:,sequence[0]]
        t=1
        while True:
            if t==T:
                break
            alpha[t]=alpha[t-1]@A*B[:,sequence[t]]
            t+=1
        return(np.sum(alpha[T-1]),alpha)
    
    def backward(A,B,pi,sequence):
        '''
        Perform the backward step in Baum-Welch Algorithm.
        
        Parameters
        ----------
        A: np.ndarray
            Stochastic transition matrix.
        B: np.ndarray
            Emission matrix.
        pi: np.1darray
            Initial state distribution.
        sequence: array-like
            The observed sequence.
            Need to be converted to integer coded.    
        '''
        N=A.shape[0]
        M=B.shape[1]
        T=len(sequence)
        beta=np.zeros([T,N])
        beta[T-1]=1
        t=T-2
        while True:
            if t<0:
                break
            beta[t]=A@(B[:,sequence[t+1]]*beta[t+1])
            t-=1
        return(np.sum(pi*B[:,sequence[0]]*beta[0]),beta)
    
    def Viterbi(A,B,pi,sequence):
        '''
        Viterbi decoding of HMM.
        
        Parameters
        ----------
        A: np.ndarray
            Stochastic transition matrix.
        B: np.ndarray
            Emission matrix.
        pi: np.1darray
            Initial state distribution.
        sequence: array-like
            The observed sequence.
            Need to be converted to integer coded.    
        '''
        N=A.shape[0]
        M=B.shape[1]
        T=len(sequence)
        delta=np.zeros([T,N])
        psi=np.zeros([T,N])
        delta[0]=pi*B[:,sequence[0]]
        t=1
        while True:
            if t==T:
                break
            delta_A=delta[t-1,np.newaxis].T*A
            delta[t]=np.max(delta_A,axis=0)*B[:,sequence[t]]
            psi[t]=np.argmax(delta_A,axis=0)
            t+=1
        psi=psi.astype(int)
        q=np.zeros(T).astype(int)
        q[T-1]=np.argmax(delta[T-1])
        t=T-2
        while True:
            if t<0:
                break
            q[t]=psi[t+1,q[t+1]]
            t-=1
        return(q)
    
    def Baum_Welch(A,B,pi,sequence,max_iter,threshold=1e-15):
        '''
        Baum-Welch algorithm of HMM. 
        See https://en.wikipedia.org/wiki/Baum%E2%80%93Welch_algorithm.
        
        Parameters
        ----------
        A: np.ndarray
            Initial stochastic transition matrix.
        B: np.ndarray
            Emission matrix.
        pi: np.1darray
            Initial state distribution.
        sequence: array-like
            The observed sequence.
            Need to be converted to integer coded.    
        '''
        N=A.shape[0]
        M=B.shape[1]
        T=len(sequence)
        likelihood,alpha=HMM.forward(A,B,pi,sequence)
        for i in range(max_iter):
            beta=HMM.backward(A,B,pi,sequence)[1]
            #temporary variables
            gamma=alpha*beta/np.sum(alpha*beta,axis=1).reshape((T,1))
            #Non-vectorized version for xi
            #xi=np.zeros([N,N,T-1])
            #for t in range(T-1):
            #    xi[:,:,t]=alpha[t].reshape((N,1))*A*beta[t+1]*B[:,sequence[t+1]]
            #    xi[:,:,t]=xi[:,:,t]/np.sum(xi[:,:,t])
            xi=alpha.T[:,np.newaxis,:-1]*A[:,:,np.newaxis]*(beta*B[:,sequence].T).T[np.newaxis,:,1:]
            xi=xi/np.sum(xi,axis=(0,1))
            pi=gamma[0]
            A=np.sum(xi,axis=2)/np.sum(gamma[:-1],axis=0).reshape([N,1])
            B=np.zeros([N,M])
            for t in range(T):
                B[:,sequence[t]]+=gamma[t]
            B=B/np.sum(gamma,axis=0).reshape([N,1])
            likelihood_new,alpha=HMM.forward(A,B,pi,sequence)
            if abs(likelihood-likelihood_new)<threshold:
                break
            likelihood=likelihood_new
        return(A,B,pi)
    
    def Baum_Welch_linear_memory(A,B,pi,sequence,max_iter,threshold=1e-15):
        '''
        Baum-Welch algorithm in linear memory.
        Implemented according to Churbanov, A., & Winters-Hilt, S. (2008).
        
        Parameters
        ----------
        A: np.ndarray
            Initial stochastic transition matrix.
        B: np.ndarray
            Emission matrix.
        pi: np.1darray
            Initial state distribution.
        sequence: array-like
            The observed sequence.
            Need to be converted to integer coded.  
        '''
        

In [187]:
M=3
N=3
pi=np.array([.3,.3,.4])
A=np.array([[.2,.3,.5],[.1,.5,.4],[.6,.1,.3]])
B=np.array([[0.1,0.5,0.4],[0.2,0.4,0.4],[0.3,0.6,0.1]])
sequence=[0,1,2,1,0]
print(HMM.forward(A,B,pi,sequence)[0])
print(HMM.forward(A,B,pi,sequence)[0])
print(HMM.Viterbi(A,B,pi,sequence))
HMM.Baum_Welch(A,B,pi,sequence,100000)

0.0030591384
0.0030591384
[2 2 0 2 2]


(array([[  5.00000000e-001,   5.00000000e-001,   3.46055935e-017],
        [  0.00000000e+000,   6.02774395e-165,   1.00000000e+000],
        [  1.00000000e+000,   0.00000000e+000,   0.00000000e+000]]),
 array([[  1.37045768e-26,   5.00000000e-01,   5.00000000e-01],
        [  2.42416509e-15,   1.00000000e+00,   0.00000000e+00],
        [  1.00000000e+00,   0.00000000e+00,   0.00000000e+00]]),
 array([ 0.,  0.,  1.]))

In [188]:
?np.sum